In [1]:
import pandas as pd
import numpy as np
import datetime
import pymongo
import base64
import time

def generateClient():
    
    return pymongo.MongoClient(base64.b64decode('bW9uZ29kYitzcnY6Ly9yZWFkb25seXVzZXI6bTRMYXJuQTdGUzhxME9KY0BneXJhbWFpcy1wcm9kdWN0aW9uLmxod3RiLm1vbmdvZGIubmV0L3Rlc3Q/cmV0cnlXcml0ZXM9dHJ1ZSZyZWFkUHJlZmVyZW5jZT1zZWNvbmRhcnkmcmVhZFByZWZlcmVuY2VUYWdzPW5vZGVUeXBlOkFOQUxZVElDUyZ3PW1ham9yaXR5').decode(), ssl=True)

In [ ]:

def main():
    
    while True:
        
        client = generateClient()
        allPartners = list(client['gyramais']['InterestFeesPartners'].aggregate([
            {
                '$match' : {
                    'document': 'PartnersGyra'
                }
            }
        ]))
        riskGroups = list(client['gyramais']['InterestFeesPartners'].aggregate([
            {
                '$match' : {
                    'document': 'RiskGroupsGyraScore'
                }
            }
        ]))

        allFees = list(client['gyramais']['InterestFeesPartners'].aggregate([
            {
                '$match' : {
                    'document': 'BusinessFees'
                }
            }
        ]))
        client.close()

        partners = pd.json_normalize(allPartners,'type')

        dfNormalized = pd.json_normalize(riskGroups,['groups','values'],[['groups','riskGroup']])

        fees = pd.json_normalize(allFees,'fees')

        for period in fees['period'].unique():
            dfNormalized.loc[dfNormalized['period'] == period,'gyraMaisFee'] = fees['gyraMaisFee'][(fees['period'] == period)].values[0]
            dfNormalized.loc[dfNormalized['period'] == period,'bankFee'] = fees['bankFee'][(fees['period'] == period)].values[0]

        df = pd.DataFrame([])
        for i in partners.iterrows():
            copy = dfNormalized.copy()
            copy.loc[:,'partnerFee'] = i[1]['partnerFee']
            copy.loc[:,'type'] = i[1]['cnpj']
            df = pd.concat([df,copy],axis=0)

        cols = []
        for col in df.columns:
            if '.' in col:
                cols.append(col.split('.')[-1])
            else:
                cols.append(col)
        df.columns = cols

        df.to_csv('gyra_fees.csv',index=False)
        
        time.sleep(43200)

if __name__ == "__main__":
    main()

In [2]:
client = generateClient()
allPartners = list(client['gyramais']['InterestFeesPartners'].aggregate([
    {
        '$match' : {
            'document': 'PartnersGyra'
        }
    }
]))
riskGroups = list(client['gyramais']['InterestFeesPartners'].aggregate([
    {
        '$match' : {
            'document': 'RiskGroupsGyraScore'
        }
    }
]))

allFees = list(client['gyramais']['InterestFeesPartners'].aggregate([
    {
        '$match' : {
            'document': 'BusinessFees'
        }
    }
]))
client.close()



In [3]:
partners = pd.json_normalize(allPartners,'type')

In [5]:
partners

,name,partnerFee,cnpj,maxInterest,minInterest,avgInterest
0,GYRA,0.0,GYRA,NaN,NaN,NaN
1,WORKMOTOR - WORKGROUP SERVICOS E COMERCIO DE P...,0.02,00213648000146,NaN,NaN,NaN
2,BRASIL FIDELIDADE BENEFICIOS LTDA.,0.02,17503933000102,NaN,NaN,NaN
3,R.T.T.L. ASSESSORIA ADMINISTRATIVA E CORRETORA...,0.01,18696927000173,NaN,NaN,NaN
4,FINPASS - F(X) ACESSO SERVICOS FINANCEIROS E D...,0.0,22359832000131,NaN,NaN,NaN
...,...,...,...,...,...,...
107,SCCRED SOLUCOES FINANCEIRAS E SEGUROS - 316579...,0.01,31657907000151,NaN,NaN,NaN
108,DAYMASTER EDICONTA GESTAO EMPRESARIAL LTDA,0.01,05996222000120,NaN,NaN,NaN
109,SAMUEL BATISTA CARVALHO DOS REIS 11982017627,0.01,41361672000100,NaN,NaN,NaN
110,LOPES - GESTAO EMPRESARIAL EIRELI,0.01,24408289000140,NaN,NaN,NaN


In [7]:
dfNormalized = pd.json_normalize(riskGroups,['groups','values'],[['groups','riskGroup']])

fees = pd.json_normalize(allFees,'fees')

for period in fees['period'].unique():
    dfNormalized.loc[dfNormalized['period'] == period,'gyraMaisFee'] = fees['gyraMaisFee'][(fees['period'] == period)].values[0]
    dfNormalized.loc[dfNormalized['period'] == period,'bankFee'] = fees['bankFee'][(fees['period'] == period)].values[0]

In [44]:
df = pd.DataFrame([])
for i in partners.iterrows():
    copy = dfNormalized.copy()
    copy.loc[:,'partnerFee'] = i[1]['partnerFee']
    copy.loc[:,'type'] = i[1]['cnpj'] + ','
    if ~np.isnan(i[1]['maxInterest']):
        copy.loc[copy['interest'] > i[1]['maxInterest'], 'interest'] = i[1]['maxInterest']
    if ~np.isnan(i[1]['minInterest']):
        copy.loc[copy['interest'] < i[1]['minInterest'], 'interest'] = i[1]['minInterest']
    df = pd.concat([df,copy],axis=0)

cols = []
for col in df.columns:
    if '.' in col:
        cols.append(col.split('.')[-1])
    else:
        cols.append(col)
df.columns = cols

df.to_csv('gyra_fees.csv',index=False)

In [29]:
partners[~partners['minInterest'].isnull()]

,name,partnerFee,cnpj,maxInterest,minInterest,avgInterest
82,Q.D. SOLUCOES E SERVICOS DE SOFTWARE S.A,0.015,29479927000138,0.0399,0.0275,0.0349
83,OLIST SERVICOS DIGITAIS LTDA,0.025,18552346000168,0.0325,0.0199,0.0249
86,LUCREE SOLUCOES INTELIGENTES S.A.,0.015,26794946000160,0.0399,0.0275,0.0349
95,KOIN ADMINISTRADORA DE CARTOES E MEIOS DE PAGA...,0.0275,17991841000100,0.0399,0.0199,0.0299


In [34]:
df[(df['type'] == '17991841000100')&(df['period'] == 1)]

,period,interest,riskGroup,gyraMaisFee,bankFee,partnerFee,type
0,1,0.0399,0,0.0405,0.0095,0.0275,17991841000100
24,1,0.0399,1,0.0405,0.0095,0.0275,17991841000100
48,1,0.0349,2,0.0405,0.0095,0.0275,17991841000100
72,1,0.0299,3,0.0405,0.0095,0.0275,17991841000100
96,1,0.0249,4,0.0405,0.0095,0.0275,17991841000100
120,1,0.0199,5,0.0405,0.0095,0.0275,17991841000100


In [35]:
df['interest'][df['type'] == '18552346000168'].mean()

0.029116666666666673

In [24]:
df.shape

(16128, 7)

In [45]:
teste = df.groupby(['period','interest','riskGroup','gyraMaisFee','bankFee','partnerFee'],as_index=False).sum()

In [46]:
teste.head()

,period,interest,riskGroup,gyraMaisFee,bankFee,partnerFee,type
0,1,0.0199,5,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
1,1,0.0199,5,0.0405,0.0095,0.01,"18696927000173,34244163000131,30598075000187,2..."
2,1,0.0199,5,0.0405,0.0095,0.015,"09196078000106,33650301000110,"
3,1,0.0199,5,0.0405,0.0095,0.02,"00213648000146,17503933000102,23690474000108,2..."
4,1,0.0199,5,0.0405,0.0095,0.025,"18552346000168,"


In [47]:
teste[teste['type'].str.contains('GYRA')]

,period,interest,riskGroup,gyraMaisFee,bankFee,partnerFee,type
0,1,0.0199,5,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
6,1,0.0249,4,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
16,1,0.0299,3,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
26,1,0.0349,2,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
38,1,0.0449,1,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
...,...,...,...,...,...,...,...
1064,24,0.0249,5,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
1074,24,0.0299,3,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
1090,24,0.0399,2,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"
1096,24,0.0499,1,0.0405,0.0095,0.0,"GYRA,22359832000131,82937293000100,"


In [59]:
df[df['type'].str.contains('29479927000138')]

,period,interest,riskGroup,gyraMaisFee,bankFee,partnerFee,type
0,1,0.0399,0,0.0405,0.0095,0.015,"29479927000138,"
1,2,0.0399,0,0.0405,0.0095,0.015,"29479927000138,"
2,3,0.0399,0,0.0405,0.0095,0.015,"29479927000138,"
3,4,0.0399,0,0.0405,0.0095,0.015,"29479927000138,"
4,5,0.0399,0,0.0405,0.0095,0.015,"29479927000138,"
...,...,...,...,...,...,...,...
139,20,0.0275,5,0.0405,0.0095,0.015,"29479927000138,"
140,21,0.0275,5,0.0405,0.0095,0.015,"29479927000138,"
141,22,0.0275,5,0.0405,0.0095,0.015,"29479927000138,"
142,23,0.0275,5,0.0405,0.0095,0.015,"29479927000138,"


In [60]:
any('22619138000106' in sub for sub in list(teste['type'].unique()))

True

In [52]:
list(teste['type'].unique())

['GYRA,22359832000131,82937293000100,',
 '18696927000173,34244163000131,30598075000187,26341030000155,38262677000170,14596052000187,33939595000102,38012186000171,31570433000106,22619138000106,36636352000185,31689620000103,34008099000190,36946097000177,23659025000105,27914090000182,34225004000190,33058052000178,39622491000148,33058052000178,10620932000190,38045664000140,06889250000100,37573613000128,37385874000114,19250309000168,34059872000147,34896052000100,21664157000191,17112767000105,33266898000101,28341296000123,38118327000135,09306604000135,32465248000114,33546628000146,30992707000192,28629250000104,31735612000156,11897817000120,31769665000198,35047953000190,37172981000164,39281210000130,38481109000160,09535727000148,18032084000100,25225706000182,15665964000126,36183091000195,22349842000196,18969415000133,29774565000108,40280209000170,33869229000116,33288665000100,22188135000165,24387319000180,26533790000164,39315420000100,38731792000147,34030055000166,26760734000162,3793927200016